In [ ]:
import os
import sys
from pathlib import Path

import pandas as pd
import plotly.express as px

sys.path.insert(0, str(Path("").resolve().parents[1]))
from utils.db_manager import MimicDBManager

In [ ]:
db_manager = MimicDBManager(port=5432)
db_manager.retrieve_table_names()

In [ ]:
[db_manager.count_rows(table_name=table) for table in db_manager.retrieve_table_names()[:10]]

In [ ]:
table_name = db_manager.retrieve_table_names()[0]
db_manager.retrieve_id(table_name, id=18207287, id_column="subject_id")

In [ ]:
pd.DataFrame.from_dict(db_manager.retrieve_column_types(table_name), orient="index", columns=["data_type"]).reset_index(
    names="column"
)

In [ ]:
pd.DataFrame.from_dict(
    {table: db_manager.retrieve_column_types(table) for table in db_manager.retrieve_table_names()}, orient="index"
).fillna("-")

In [ ]:
data = db_manager.retrieve_all("mimiciv_hosp.diagnoses_icd")
data["icd_code"] = data["icd_code"].str.strip()
aki = data[data.icd_code == "5848"]
aki

In [ ]:
patients = db_manager.retrieve_all("mimiciv_hosp.patients")
aki_patients = aki.merge(patients, on="subject_id")
aki_patients

In [ ]:
fig = px.histogram(aki_patients, x="anchor_age", color="gender", barmode="overlay", marginal="box", text_auto=True)
fig.show()